In [ ]:
import ROOT
# from T2KStyle import T2KStyle
c = ROOT.TCanvas('c', '', 800, 600)
cw = ROOT.TCanvas('c2', '', 1600, 600)
cl = ROOT.TCanvas('cl', '', 1600, 1200)

In [ ]:
# path = '/eos/user/s/ssuvorov/DESY_testbeam/cosmic_strict_v2/'
path = '~/LXPLUS/DESY_testbeam/tree/'
voltage = '360'

file_name = [path + '/g_' + voltage + '_200_100p',   # 0
             path + '/g_' + voltage + '_200_95p',    # 1
             path + '/g_' + voltage + '_200_90p',    # 2
             path + '/g_' + voltage + '_200_80p',    # 3
             path + '/g_' + voltage + '_200_3000',   # 4
             path + '/g_' + voltage + '_200_2000',   # 5
             path + '/g_' + voltage + '_200_95p_nbr', # 6
             path + '/g_' + voltage + '_200_90p_nbr'] # 7

title = ['100%', '95%', '90%', '80%', 'Q < 3000', 'Q < 2000', '95% - high charge neighbours', '90% - high charge neighbours']
color = [ROOT.kRed, ROOT.kGreen, ROOT.kBlack, ROOT.kBlue, ROOT.kViolet, ROOT.kYellow, ROOT.kCyan, ROOT.kOrange, ROOT.kMagenta, ROOT.kAzure]
N_iter = 1

## Resolution vs charge cuts

In [ ]:
res_evo = []
mg = ROOT.TMultiGraph()
N_iter = 10
for f in [0, 1, 2, 3, 4, 5, 6, 7]:
    res_evo.append(ROOT.TGraphErrors())
    for it in range(N_iter):
        file = ROOT.TFile(file_name[f] + '_iter' + str(it) + '.root')
        res = file.Get('resol_total')
        res.Fit('gaus', 'Q')
        fit_res = res.GetFunction("gaus");
        sigma = fit_res.GetParameter(2);
        sigma_e = fit_res.GetParError(2);
        res_evo[-1].SetPoint(it, it, sigma*1e6)
        res_evo[-1].SetPointError(it-1, sigma_e) 
    res_evo[-1].SetTitle(title[f])
    res_evo[-1].SetMarkerColor(color[f])
    mg.Add(res_evo[-1])

In [ ]:
%jsroot on
c.Clear()
mg.GetXaxis().SetRangeUser(-1, 10)
mg.GetXaxis().SetTitle('Iteration')
mg.GetYaxis().SetTitle('Resolution [#mum]')
mg.SetMinimum(0.)
mg.SetMaximum(500.)
mg.Draw('ap')
c.BuildLegend()
c.SetGrid()
c.Draw()

## Residuals
Compare residuals for CoC and PRF for two files

In [ ]:
cw.cd()
cw.Clear()
cw.Divide(2)

N_iter = 6

ROOT.gStyle.SetOptTitle(1)
it = [2, N_iter-1]
color = [ROOT.kBlack, ROOT.kRed]
hs = []
res = []
file = []

ci = 1
# files from the list
for f in [0, 4]:
    cw.cd(ci)
    ci += 1
    hs.append(ROOT.THStack('hs', title[f]))
    for i in range(2):
        file.append(ROOT.TFile(file_name[f] + '_iter' + str(it[i]) + '.root'))
        res.append(file[-1].Get('resol_total'))
        res[-1].SetLineColor(color[i])
        if i != 0:
            res[-1].Fit('gaus', 'Q')
            print(res[-1].GetMean(), res[-1].GetFunction('gaus').GetParameter(1))
        hs[-1].Add(res[-1])
    hs[-1].Draw('nostack')
    hs[-1].GetXaxis().SetRangeUser(-2e-3, 2e-3)
cw.Draw()        

## Charge vs residuals

In [ ]:
%jsroot on
it = 9
c.cd()
file = ROOT.TFile(path + '../tree/g_' + voltage + '_200_1800_iter' + str(it) + '.root')
h = ROOT.TH2F('h', '', 200, -2., 2., 100, 0., 7000)
file.outtree.Project('h', 'charge:1e3*residual', '')
h.GetXaxis().SetTitle('Residual [mm]')
h.GetYaxis().SetTitle('Charge [a.u.]')
h.Draw('colz')
c.Draw()

## Spatial resolution tree reader
Plots track residuals; residuals vs charge; bias; number of clusters per track

In [ ]:
it = 3
file = ROOT.TFile(path + '/g_' + voltage + '_200_3000_iter' + str(it) + '.root')
cl.cd()
cl.Clear()
cl.Divide(2, 2)
h = ROOT.TH1F('h', '', 200, 0., 2.)
hm = ROOT.TH1F('hm', '', 200, 0., 1e-4)
h2 = ROOT.TH2F('h2', '', 200, 0., 2., 100, 0., 10000.)
hc = ROOT.TH1F('hc', '', 36, 0., 36.)
for entry in file.outtree:
    mean = 0
    n = 0
    max_q = 0
    dedx = 0
    for i in range(len(entry.charge)):
        res = entry.residual[i]
        if res == -999:
            continue
        mean += entry.residual[i]*1e3
        n += 1
        dedx += entry.charge[i]
        if entry.charge[i] > max_q:
            max_q = entry.charge[i]
    if n == 0:
        print('zero n')
        continue
    mean /= n
    rms = 0
    for i in range(len(entry.charge)):
        res = entry.residual[i]
        if res == -999:
            continue
        rms += (res*1e3 - mean)**2
    rms /= n**0.5
    dedx /= n

    if (rms != 0):
        h.Fill(rms)
        hm.Fill(mean)
        h2.Fill(rms, max_q)
        hc.Fill(n)
        
cl.cd(1)
h.GetXaxis().SetTitle('Track residual RMS [mm]')
h.Draw('')
cl.cd(2)
h2.GetXaxis().SetTitle('Track residual RMS [mm]')
h2.GetYaxis().SetTitle('Max charge in track [a.u.]')
h2.Draw('colz')
cl.cd(3)
hm.GetXaxis().SetTitle('Track residual mean [mm]')
hm.Draw('')
cl.cd(4)
hc.GetXaxis().SetTitle("clusters used")
hc.Draw()

cl.Draw()

## PRF plotter
plots PRF with various cuts on e.g. charge

In [ ]:
cw.cd()
cw.Clear()
cw.Divide(2)
cw.cd(1)
h = ROOT.TH2F('h', '', 200, -4., 4., 100, 0., 1.)
h.SetTitle('No charge limit')
file.outtree.Project('h', 'qfrac:1e2*dx', '')
h.GetXaxis().SetTitle('Residual [cm]')
h.GetYaxis().SetTitle('Q_{pad}/Q_{cluster}')
h.Draw('colz')
cw.cd(2)
h2 = ROOT.TH2F('h2', '', 200, -4., 4., 100, 0., 1.)
h2.SetTitle('Q_{cluster} < 2000 c.u.')
file.outtree.Project('h2', 'qfrac:1e2*dx', 'charge < 2000')
h2.GetXaxis().SetTitle('Residual [cm]')
h2.GetYaxis().SetTitle('Q_{pad}/Q_{cluster}')
h2.Draw('colz')
cw.Draw()

## Track bias vs column

In [ ]:
c.cd()
gr = ROOT.TGraph()
mean = [0]*36
n = [0]*36
m = 0.
mn = 0
h = ROOT.TH1F('h', '', 1000, -2., 2.)
for entry in file.outtree:
    for col in range(len(entry.residual)):
        res = 1e3*entry.residual[col]
        if res != -999 and abs(res) < 2:
            h.Fill(res)
            mean[col] += res
            n[col] += 1
            m += res
            mn += 1
            if m / mn > 1:
                print("ala", res, m, mn, m / mn)
                break

print(m / mn)
for col in range(36):
    if n[col] != 0:
        gr.SetPoint(gr.GetN(), col, mean[col]/n[col])
gr.Draw('apl')
gr.GetYaxis().SetTitle('Bias [mm]')
c.Draw()

## Residual anomalies
Plot abnormal residuals. Specify the rms threshold and change the number of anomaly you want to look at `if found > 32:`. It will dump the id of event. It can be looked later with EventDisplay

In [ ]:
cw.cd()
cw.Clear()
cw.Divide(2)
it = 4
rms_threshold = .4
file = ROOT.TFile(path + '/g_' + voltage + '_200_90p_iter' + str(it) + '.root')
h = ROOT.TH1F('h', '', 36, 0., 36.)
hc = ROOT.TH1F('hc', '', 36, 0., 36.)
found = 0
for entry in file.outtree:
    mean = 0
    n = 0
    max_q = 0
    dedx = 0
    for i in range(len(entry.charge)):
        res = entry.residual[i]
        if res == -999:
            continue
        mean += entry.residual[i]*1e3
        n += 1
        dedx += entry.charge[i]
        if entry.charge[i] > max_q:
            max_q = entry.charge[i]
    if n == 0:
        print('zero n')
        continue
    mean /= n
    rms = 0
    for i in range(len(entry.charge)):
        res = entry.residual[i]
        if res == -999:
            continue
        rms += (res*1e3 - mean)**2
    rms /= n**0.5
    dedx /= n
    if rms > rms_threshold:
        h.Reset()
        hc.Reset()
        for i in range(len(entry.charge)):
            if ( entry.residual[i] != -999):
                h.SetBinContent(i+1, entry.residual[i]*1e3)
                hc.SetBinContent(i+1, entry.charge[i])
        found += 1
    if found > 32:
        print(entry.ev)
        break

cw.cd(1)
h.GetYaxis().SetTitle('Residual [mm]')
h.Draw('hist')
cw.cd(2)
hc.GetYaxis().SetTitle('Charge [a.u.]')
hc.Draw('hist')
cw.Draw()

# OLD

In [ ]:
res_evo = []

# path = '~/LXPLUS/DESY_testbeam/cosmic_strict_v2/'
path = '/eos/user/s/ssuvorov/DESY_testbeam/cosmic_strict_v2/'

file_name = [path + '/mm1_380V_200ns',
            path + '/er2_380V',
            path + '/er3_380V',
            path + '../v12/g_380_200',
            path + '../cosmic_test/c_0d_0T_200ns_25MHz']

title = ['MM1', 'ERAM2', 'ERAM3', 'DESY beam', 'DESY cosmic']
color = [ROOT.kRed, ROOT.kGreen, ROOT.kBlue, ROOT.kBlack, ROOT.kViolet]
N_iter = 7

mg = ROOT.TMultiGraph()

for f in [0, 1, 2, 3]:
    for it in range(N_iter):
        file = ROOT.TFile(file_name[f] + '_iter' + str(it) + '.root')
        res = file.Get('resol_total')
        res.Fit('gaus', 'Q')
        fit_res = res.GetFunction("gaus");
        sigma = fit_res.GetParameter(2);
        sigma_e = fit_res.GetParError(2);
        res_evo.append(ROOT.TGraphErrors())
        res_evo[f].SetPoint(it, it, sigma*1e6)
        res_evo[f].SetPointError(it-1, sigma_e) 
    res_evo[f].SetTitle(title[f])
    res_evo[f].SetMarkerColor(color[f])
    res_evo[f].SetLineColor(color[f])
    mg.Add(res_evo[f])

In [ ]:
%jsroot on
c.Clear()
mg.GetXaxis().SetRangeUser(-1, 7)
mg.GetXaxis().SetTitle('Iteration')
mg.GetYaxis().SetTitle('Resolution [#mum]')
mg.SetMinimum(0.)
mg.SetMaximum(1300.)
mg.Draw('apl')
c.BuildLegend()
c.SetGrid()
c.Draw()

In [ ]:
c.Clear()

file_name = path + '../cosmic_strict/mm1_360V_200ns'

file_1 = ROOT.TFile(file_name + '_iter' + str(0) + '.root')
h_1 = file_1.Get('resol_total')

# file_name = ROOT.TFile(file_name + '_iter' + str(N_iter-1) + '.root')
# N_iter = 18


file_2 = ROOT.TFile(file_name + '_iter' + str(N_iter-1) + '.root')
h_2 = file_2.Get('resol_total')
PRF_2 = file_2.Get('PRF_histo')

mean = h_2.GetMean()
max = h_2.GetMaximum()
start = h_2.GetBinLowEdge(h_2.FindFirstBinAbove(max/2))
end = h_2.GetBinLowEdge(h_2.FindLastBinAbove(max/2)) + h_2.GetBinWidth(h_2.FindLastBinAbove(max/2))

print(end-start)

h_2.Fit('gaus', 'Q', '', mean-4*sigma, mean+4*sigma)
print(h_2.GetFunction('gaus').GetParameter(2) * 1e6, 'um')

h_2.SetLineColor(ROOT.kRed)

h_2.Draw('')
h_1.Draw('same')
c.Draw()

In [ ]:
c.Clear()


file_1 = ROOT.TFile(file_name + '_iter' + str(0) + '.root')
h_1 = file_1.Get('resol_total')

# file_name = ROOT.TFile(file_name + '_iter' + str(N_iter-1) + '.root')
# N_iter = 18


file_2 = ROOT.TFile(file_name + '_iter' + str(N_iter-1) + '.root')
h_2 = file_2.Get('resol_total')
PRF_2 = file_2.Get('PRF_histo')

mean = h_2.GetMean()
max = h_2.GetMaximum()
start = h_2.GetBinLowEdge(h_2.FindFirstBinAbove(max/2))
end = h_2.GetBinLowEdge(h_2.FindLastBinAbove(max/2)) + h_2.GetBinWidth(h_2.FindLastBinAbove(max/2))

print(end-start)

h_2.Fit('gaus', 'Q', '', mean-4*sigma, mean+4*sigma)
print(h_2.GetFunction('gaus').GetParameter(2) * 1e6, 'um')

h_2.SetLineColor(ROOT.kRed)
h_2.Draw()
h_1.Draw('same')
c.Draw()

In [ ]:
c.Clear()
c.cd()

file_name1 = path + '../cosmic_eram/mm1_380V_200ns_iter5.root'
file_name2 = path + '../test/mm1_380V_200ns_v4_iter5.root'
file1 = ROOT.TFile(file_name1, 'READ')
file2 = ROOT.TFile(file_name2, 'READ')

h1 = file1.Get('resol_total')
h2 = file2.Get('resol_total')
# h1.Fit('gaus')

h2.SetLineColor(ROOT.kGreen)

h1.Draw()
h2.Draw('same')
c.Draw()